In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicLayer(nn.Module):
    def __init__(self, input_features, output_features, init_weights=None, init_bias=None):
        super(DynamicLayer, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        if init_weights is not None:
            self.weights = nn.Parameter(init_weights)
        else:
            self.weights = nn.Parameter(torch.randn(output_features, input_features))

        if init_bias is not None:
            self.bias = nn.Parameter(init_bias)
        else:
            self.bias = nn.Parameter(torch.randn(output_features))
    def grow_neurons(self, new_output_features):
        # 初始化新增加的权重和偏置为0
        additional_weights = nn.Parameter(torch.zeros(new_output_features, self.input_features))
        additional_bias = nn.Parameter(torch.zeros(new_output_features))
        self.weights = nn.Parameter(torch.cat([self.weights, additional_weights], dim=0))
        self.bias = nn.Parameter(torch.cat([self.bias, additional_bias]))
        self.output_features += new_output_features
    def adjust_input_dim(self, new_input_features):
        # 初始化新增加的权重为0
        additional_weights = nn.Parameter(torch.zeros(self.output_features, new_input_features - self.input_features))
        self.weights = nn.Parameter(torch.cat([self.weights, additional_weights], dim=1))
        self.input_features = new_input_features

    def forward(self, x):
        return F.linear(x, self.weights, self.bias)

class FlexibleNN(nn.Module):
    def __init__(self):
        super(FlexibleNN, self).__init__()
        # 初始化权重和偏置
        init_weights1 = torch.randn(5, 10)
        init_bias1 = torch.randn(5)
        init_weights2 = torch.randn(3, 5)
        init_bias2 = torch.randn(3)

        self.layer1 = DynamicLayer(10, 5, init_weights=init_weights1, init_bias=init_bias1)
        self.layer2 = DynamicLayer(5, 3, init_weights=init_weights2, init_bias=init_bias2)

    def grow_network(self):
        # 在第一层增加3个神经元
        self.layer1.grow_neurons(3)
        # 调整第二层以匹配第一层的新输出维度
        self.layer2.adjust_input_dim(self.layer1.output_features)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x




In [28]:
# 示例使用
net = FlexibleNN()
input_data = torch.randn(1, 10)
output = net(input_data)

print("初始状态的权重:")
print("第一层权重:", net.layer1.weights.data)
print("第二层权重:", net.layer2.weights.data)

初始状态的权重:
第一层权重: tensor([[ 0.8004, -0.0968, -0.5970, -0.5206, -0.8678, -0.0295,  0.2575, -0.2446,
          0.0412, -0.1789],
        [ 0.2142,  0.3899, -1.4569, -0.3474,  1.7738,  0.2485,  1.2093,  0.1862,
         -1.1474, -1.8927],
        [ 1.5338,  0.9287, -1.7201,  1.7727,  1.4645, -1.6267, -1.2569, -0.7487,
          0.9750, -1.1398],
        [ 0.5080,  0.3049, -0.2937, -0.3996,  0.6779,  0.2909,  0.2435,  0.2926,
         -0.3535, -0.5695],
        [ 0.5954, -0.1051,  0.6220, -1.0150,  0.1609,  0.2735,  0.0149,  0.3356,
         -0.1692, -1.0205]])
第二层权重: tensor([[ 0.8691,  1.3269,  0.2163, -0.7423,  1.4990],
        [-0.6174,  2.0788,  1.9671, -0.3645,  1.9890],
        [-0.9785, -1.2055,  1.3217,  1.1798, -0.2769]])


In [29]:
output

tensor([[-10.6527, -31.4744,  -3.7672]], grad_fn=<AddmmBackward0>)

In [30]:
# 网络增长
net.grow_network()

# 再次使用相同的输入
output = net(input_data)
output


tensor([[-10.6527, -31.4744,  -3.7672]], grad_fn=<AddmmBackward0>)

In [31]:
print("\n增长后的权重:")
print("第一层权重:", net.layer1.weights.data)
print("第二层权重:", net.layer2.weights.data)


增长后的权重:
第一层权重: tensor([[ 0.8004, -0.0968, -0.5970, -0.5206, -0.8678, -0.0295,  0.2575, -0.2446,
          0.0412, -0.1789],
        [ 0.2142,  0.3899, -1.4569, -0.3474,  1.7738,  0.2485,  1.2093,  0.1862,
         -1.1474, -1.8927],
        [ 1.5338,  0.9287, -1.7201,  1.7727,  1.4645, -1.6267, -1.2569, -0.7487,
          0.9750, -1.1398],
        [ 0.5080,  0.3049, -0.2937, -0.3996,  0.6779,  0.2909,  0.2435,  0.2926,
         -0.3535, -0.5695],
        [ 0.5954, -0.1051,  0.6220, -1.0150,  0.1609,  0.2735,  0.0149,  0.3356,
         -0.1692, -1.0205],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000]])
第二层权重: tensor([[ 0.8691,  1.3269,  0.2163, -0.7423,  1.4990,  0.0000,  0.0000,  0.0000],
        [-0.6174,  2.0